In [0]:
%pip install  thunder-python



Python interpreter will be restarted.
  Created wheel for thunder-python: filename=thunder_python-1.4.2-py3-none-any.whl size=43290 sha256=4ae1508c05eedb86abd8cccda8cc986b219a326f5cb6053dd0671282b165aca1
  Stored in directory: /root/.cache/pip/wheels/2a/1d/be/09ebadbc2f4d32932614d81c076239ef1974a311e03f018cdf
  Created wheel for bolt-python: filename=bolt_python-0.7.1-py3-none-any.whl size=31180 sha256=2d0b950dd6ce75dd19a47a87a0d2b49e5b662c7ff03b6a97b83840dfa1a4c57c
  Stored in directory: /root/.cache/pip/wheels/73/4f/ea/b613f2e93096c03fad1d93caba8bfa42aa81df0f85329cb0b6
Successfully built thunder-python bolt-python
Python interpreter will be restarted.


In [0]:
# coding=utf-8

# Copyright 2015 and onwards Sanford Ryza, Uri Laserson, Sean Owen and Joshua Wills
#
# See LICENSE file for further information.

# this code assumes you are working from an interactive Thunder (PySpark) shell

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

import thunder as td

##################
# data exploration/Trabalho de Computação Paralela/fish_dataset/tif-stack/time001.tif
##################

# load some zebrafish brain data
data = td.images.fromtif('./fish_dataset/', engine=sc)

# explore the resulting object
print(data)
print(data.values)
print(data.values.first())
print(data.values._rdd)
print(data.values._rdd.first())
print(data.shape)
print(data.count())

# plot the raw data
img = data.first()
plt.imshow(img[:, : ,0], interpolation='nearest', aspect='equal', cmap='gray')

# plot subsampled data
subsampled = data.subsample((5, 5, 1))
plt.imshow(subsampled.first()[1][:, : ,0], interpolation='nearest', aspect='equal', cmap='gray')
print(subsampled.shape)

# reshuffle data to series representation
series = data.toseries()
print(series.shape)
print(series.index)
print(series.count())
print(series.values._rdd.takeSample(False, 1)[0])
print(series.max().values)

# distributed computation of stats
stddev = series.map(lambda s: s.std())
print(stddev.values._rdd.take(3))
print(stddev.shape)

# collecting data locally and repacking it
repacked = stddev.toarray()
plt.imshow(repacked[:,:,0], interpolation='nearest', cmap='gray', aspect='equal')
print(type(repacked))
print(repacked.shape)

# plot some of the time series themselves
plt.plot(series.center().sample(50).toarray().T)

# distributed computatino of custom statistics
series.map(lambda x: x.argmin())


---------------------------------------------------------------------------
FileNotFoundError                         Traceback (most recent call last)
File <command-1092222183704395>:20
     13 import thunder as td
     15 ##################
     16 # data exploration/Trabalho de Computação Paralela/fish_dataset/tif-stack/time001.tif
     17 ##################
     18 
     19 # load some zebrafish brain data
---> 20 data = td.images.fromtif('./fish_dataset/time001.tif/', engine=sc)
     22 # explore the resulting object
     23 print(data)

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-2991b55e-26e7-4e01-8829-8957396c1c2c/lib/python3.9/site-packages/thunder/images/readers.py:392, in fromtif(path, ext, start, stop, recursive, nplanes, npartitions, labels, engine, credentials, discard_extra)
    389     return zip(keys, values)
    391 recount = False if nplanes is None else True
--> 392 return frompath(path, accessor=getarray, ext=ext, start=start, stop=stop,
    393                

In [0]:


###############################
# Clustering fish brain regions
###############################
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns

import thunder as td
import numpy as np
from pyspark.mllib.clustering import KMeans

images = td.images.frombinary(
    '/fish-long', order='F', engine=sc,  conf='conf.json')
series = images.toseries()

normalized = series.normalize(method='mean')
stddevs = (normalized
    .map(lambda s: s.std())
    .sample(1000))
plt.hist(stddevs.values, bins=20)
plt.plot(
    normalized
        .filter(lambda s: s.std() >= 0.1)
        .sample(50)
        .values.T)

# perform k-means on the normalized series
ks = [5, 10, 15, 20, 30, 50, 100, 200]
models = []
for k in ks:
    models.append(KMeans.train(normalized.values._rdd.values(), k))

# define a couple functions to score the clustering quality
def model_error_1(model):
    def series_error(series):
        cluster_id = model.predict(series)
        center = model.centers[cluster_id]
        diff = center - series
        return diff.dot(diff) ** 0.5

    return (normalized
        .map(series_error)
        .toarray()
        .sum())

def model_error_2(model):
    return model.computeCost(normalized.values._rdd.values())

# compute the error metrics for the different resulting clusterings
errors_1 = np.asarray(map(model_error_1, models))
errors_2 = np.asarray(map(model_error_2, models))
plt.plot(
    ks, errors_1 / errors_1.sum(), 'k-o',
    ks, errors_2 / errors_2.sum(), 'b:v')

# plot the best performing model
model20 = models[3]
plt.plot(np.asarray(model20.centers).T)

# finally, plot each brain region according to its characteristic behavior
from matplotlib.colors import ListedColormap
cmap_cat = ListedColormap(sns.color_palette("hls", 10), name='from_list')
by_cluster = normalized.map(lambda s: model20.predict(s)).toarray()
plt.imshow(by_cluster[:, :, 0], interpolation='nearest',
    aspect='equal', cmap='gray')

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File <command-2953301531535851>:12
      9 import numpy as np
     10 from pyspark.mllib.clustering import KMeans
---> 12 images = td.images.frombinary(
     13     '/fish-long', order='F', engine=sc,  conf='conf.json')
     14 series = images.toseries()
     16 normalized = series.normalize(method='mean')

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-2991b55e-26e7-4e01-8829-8957396c1c2c/lib/python3.9/site-packages/thunder/images/readers.py:279, in frombinary(path, shape, dtype, ext, start, stop, recursive, nplanes, npartitions, labels, conf, order, engine, credentials)
    276     shape = params['shape']
    278 if not shape:
--> 279     raise ValueError('Image shape must be specified as argument or in a conf.json file')
    281 if not dtype:
    282     dtype = 'int16'

ValueError: Image shape must be specified as argument or in a 

In [0]:
import thunder

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-261437857634653>:1
----> 1 import thunder

File /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171, in _create_import_patch.<locals>.import_patch(name, globals, locals, fromlist, level)
    166 thread_local._nest_level += 1
    168 try:
    169     # Import the desired module. If you’re seeing this while debugging a failed import,
    170     # look at preceding stack frames for relevant error information.
--> 171     original_result = python_builtin_import(name, globals, locals, fromlist, level)
    173     is_root_import = thread_local._nest_level == 1
    174     # `level` represents the number of leading dots in a relative import statement.
    175     # If it's zero, then this is an absolute import.

ModuleNotFoundError: No module named 'thunder'